In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the dataset
data = open('1661-0.txt', encoding='utf-8').read()

# Preprocess the text data
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for equal input length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split sequences into input and label
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
# Train the model
history = model.fit(xs, ys, epochs=10, verbose=1)

Epoch 1/10
3176/3176 [==============================] - 138s 42ms/step - loss: 6.1956 - accuracy: 0.0814
Epoch 2/10
3176/3176 [==============================] - 175s 55ms/step - loss: 5.4282 - accuracy: 0.1334
Epoch 3/10
3176/3176 [==============================] - 197s 62ms/step - loss: 5.0123 - accuracy: 0.1582
Epoch 4/10
3176/3176 [==============================] - 171s 54ms/step - loss: 4.6693 - accuracy: 0.1775
Epoch 5/10
3176/3176 [==============================] - 148s 47ms/step - loss: 4.3508 - accuracy: 0.1974
Epoch 6/10
3176/3176 [==============================] - 141s 44ms/step - loss: 4.0434 - accuracy: 0.2217
Epoch 7/10
3176/3176 [==============================] - 133s 42ms/step - loss: 3.7474 - accuracy: 0.2522
Epoch 8/10
3176/3176 [==============================] - 145s 46ms/step - loss: 3.4670 - accuracy: 0.2886
Epoch 9/10
3176/3176 [==============================] - 139s 44ms/step - loss: 3.2022 - accuracy: 0.3280
Epoch 10/10
3176/3176 [==============================] 

In [5]:
predicted_probabilities = model.predict(token_list, verbose=0)
predicted_index = np.argmax(predicted_probabilities)
output_word = ""

for word, index in tokenizer.word_index.items():
    if index == predicted_index:
        output_word = word
        break

seed_text += " " + output_word
print("Next words:", seed_text)

Next words: The quick brown ones
